In [7]:
import pathlib
from pathlib import Path
import pandas as pd
import xarray as xr

In [8]:
def process_chromeleon_file(filepath: pathlib.PosixPath, tmin: float, tmax: float):
    with filepath.open() as f: 
        lines = f.readlines()
   
    lines = [line.strip().split('\t') for line in lines]
    lines = lines[44:-1]
    data = pd.DataFrame(lines, columns = ['t', 'step', 's'], dtype='float')
    data = data.drop(columns='step')
    data['ds'] = data['s'].diff()
    data['d2s'] = data['ds'].diff()
    data = data.where(data['t'] > tmin).where(data['t'] < tmax).dropna()
    
    return data

def get_fanny_concentration(file: pathlib.PosixPath):
    n = file.name
    conc = {'1':0.05, '2':0.1, '3': 0.2, '4':0.3, '5': 0.4, '6':0.5}
    return conc[n[1]]

def read_calib(p: pathlib.PosixPath, tmin=0, tmax=30):
    files = p.glob('*.txt')
    acid = p.name.split(' ')[0].lower().capitalize()
    print(acid)
    data_list = []
    for file in files:
        data = process_chromeleon_file(file, tmin, tmax)
        data[acid] = get_fanny_concentration(file)
        data = data.set_index(['t', acid]).to_xarray()
        data_list.append(data)
    data = xr.combine_by_coords(data_list)
    savepath = str(p / acid) + '.nc'
    data.to_netcdf(savepath)
    return files, data

In [9]:
path = Path('/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/')
path = path.resolve()
print(path)
acids = [i for i in path.iterdir() if i.is_dir()]
for acid in acids:
    print(acid)
    files, data = read_calib(acid, tmin=6.5, tmax=9)

/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/ACETIC ACID
Acetic
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/BUTYRIC ACID
Butyric
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/PROPANOIC ACID
Propanoic
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/VALERIC ACID
Valeric
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves/CAPROIC ACID
Caproic


In [10]:
tmin = 6.5
tmax = 9
a_path, ace = read_calib(acids[0],  tmin=tmin, tmax=tmax)
p_path, pro = read_calib(acids[1],  tmin=tmin, tmax=tmax)
b_path, but = read_calib(acids[2], tmin=tmin, tmax=tmax)
v_path, val = read_calib(acids[3], tmin=tmin, tmax=tmax)

Acetic
Butyric
Propanoic
Valeric
